In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import os
import utils
plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)
pd.set_option('display.float_format', lambda x: '%.10f' % x)

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn


In [3]:
splits = utils.get_train_val_test_split()

X_train, y_train = splits['train']['X'], splits['train']['y']
X_val, y_val = splits['val']['X'], splits['val']['y']
X_test, y_test = splits['test']['X'], splits['test']['y']
y_train.dtype


dtype('float64')

In [10]:
from typing import Tuple


class SuicideDataSet(Dataset):
    def __init__(self, split: str = 'train'):
        self.split = split
        self.splits = utils.get_train_val_test_split()

    def __len__(self):
        return len(self.splits[self.split]['y'])

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        ftrs = self.splits[self.split]['X'][idx]
        label = self.splits[self.split]['y'][idx]
        ftrs = torch.tensor(ftrs).float()
        label = torch.tensor(label).float()
        return ftrs, label


In [11]:
class SuicideRegressor(nn.Module):
    def __init__(self, in_features) -> None:
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(in_features,512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,64),
            nn.ReLU(),
            nn.Linear(64,8),
            nn.ReLU(),
            nn.Linear(8,1),
        )

    def forward(self, x):
        return self.model(x)

In [12]:
class SuicideRegressorBN(nn.Module):
    def __init__(self, in_features) -> None:
        super().__init__()
        self.model = nn.Sequential(
            nn.BatchNorm1d(num_features=in_features),
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=512),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=256),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=64),
            nn.Linear(64, 8),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=8),
            nn.Linear(8, 1),
        )

    def forward(self, x):
        return self.model(x)

## Train model

In [13]:
train_ds = SuicideDataSet(split='train')
train_loader = DataLoader(dataset=train_ds, batch_size=1788)

val_ds = SuicideDataSet(split='val')
val_loader = DataLoader(dataset=val_ds, batch_size=276)

a = iter(train_loader)
b = a._next_data()
print(b)
# print(b)


[tensor([[-0.3843, -0.1995,  1.0116,  ..., -0.4817, -0.3738, -0.5036],
        [ 0.3546,  1.0578,  1.0116,  ..., -1.3187,  2.2703,  0.7375],
        [-1.7566,  2.3151, -0.9886,  ...,  0.5982,  0.2023,  0.0966],
        ...,
        [ 1.1991, -0.1995, -0.9886,  ...,  0.5982, -0.8465, -0.4315],
        [-0.0676, -1.4568, -0.9886,  ...,  0.5982,  0.9557, -0.4183],
        [-1.2288,  3.5723, -0.9886,  ..., -1.1297,  3.1419,  0.3082]]), tensor([100.1000,   9.8300,   7.2800,  ...,   4.6500,   3.7200,   4.5000])]


In [14]:
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
writer = SummaryWriter(log_dir=f"./tb_logs/{datetime.now().strftime('%Y-%M-%d_%H:%m')}")



In [15]:

EPOCHS = 1_000
model = SuicideRegressorBN(in_features=9)
criterion = nn.MSELoss()
optim = torch.optim.Adam(
    params=model.parameters(),
    lr=5e-3,
    weight_decay=1e-4
)

for epoch in range(EPOCHS):
    model.train()
    loss = 0
    for batch_ftrs, batch_labels in train_loader:
        optim.zero_grad()
        y_pred = model(batch_ftrs)
        train_loss = criterion(y_pred[:,0], batch_labels)
        train_loss.backward()

        optim.step()
        loss += train_loss.item()

    loss = loss / len(train_loader)

    # compute validation loss
    val_loss = 0
    with torch.no_grad():
        model.eval()
        for batch_ftrs, batch_labels in val_loader:
            y_pred = model(batch_ftrs)
            cur_loss = criterion(y_pred[:,0], batch_labels)
            val_loss += cur_loss.item()
        val_loss = val_loss / len(val_loader)
    # prints
    if epoch % 200 == 0:
        print("epoch : {}/{}, train_loss = {:.6f}".format(epoch + 1, EPOCHS, loss))
        print("epoch : {}/{}, val_loss = {:.6f}".format(epoch + 1, EPOCHS, val_loss))
     # add scalars to tensorboard
    writer.add_scalar(f"{criterion}/train", loss, epoch + 1)
    writer.add_scalar(f"{criterion}/val", val_loss, epoch + 1)
    writer.add_scalars("comp", {'train':loss, 'val':val_loss}, epoch+1)

    writer.flush()

epoch : 1/3000, train_loss = 454.652771
epoch : 1/3000, val_loss = 494.151917
epoch : 201/3000, train_loss = 9.371614
epoch : 201/3000, val_loss = 16.167332
epoch : 401/3000, train_loss = 0.109246
epoch : 401/3000, val_loss = 7.856324
epoch : 601/3000, train_loss = 0.052403
epoch : 601/3000, val_loss = 8.351795
epoch : 801/3000, train_loss = 0.038651
epoch : 801/3000, val_loss = 8.012628
epoch : 1001/3000, train_loss = 0.064795
epoch : 1001/3000, val_loss = 7.813686
epoch : 1201/3000, train_loss = 0.177920
epoch : 1201/3000, val_loss = 7.703077
epoch : 1401/3000, train_loss = 0.036813
epoch : 1401/3000, val_loss = 7.966930


KeyboardInterrupt: 

In [ ]:
torch.save(model, "torch_models/bn_1.pickle")

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error

with torch.no_grad():
    model.eval()
    pred = model(torch.tensor(X_test_std).float())
    custom_mlp_pred = pred[:, 0]


## compare with sklearn MLP

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp_model = MLPRegressor(
    hidden_layer_sizes=[512,256,64,8],
    max_iter=3000,
    activation='relu')
mlp_model.fit(X_train_std,Y_train)

In [ ]:
mlp_pred = mlp_model.predict(X_test_std)
metrics = [mean_squared_error, mean_absolute_error, median_absolute_error]
model_preds = {
    'mlp': mlp_pred,
    'custom_mlp': custom_mlp_pred
}


for metric in metrics:
    print(f"{metric.__name__} eval:")
    for model, pred in model_preds.items():
        score = metric(y_true=Y_test, y_pred=pred)
        print(f"{model}: {score}")
    print('-'*10)

In [ ]:
n_samples = 50
pred_df = pd.DataFrame({
    'sklearn_mlp_pred': mlp_model.predict(X_test_std)[:n_samples],
    'custom_mlp_pred' : custom_mlp_pred[:n_samples],
    'ground_truth': Y_test[:n_samples]
})
sns.scatterplot(pred_df, markers=True, alpha=.6)
sns.lineplot(pred_df)